In [1]:
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
# In order to use this notebook for univarate time series analysis :-
# 1) The primary requirement is not to have missing values or categorial(string) data for time_dependent variable 
#    and time_column.
# 2) This cell requires information on file_name (only csv), time_dependent_variable, time_column, date_time format (frmt)
#    and resample grain(X). After filling the required information correctly, you can run all the cells (Cell ---> Run All)
# 3) Example :-
#   file_name               = "JetRail Avg Hourly Traffic Data - 2012-2013.csv"
#   time_dependent_variable = "Count"    (column name in your dataset)
#   time_column             = "Datetime" (column name in your dataset)
#   frmt                    = "%Y-%m-%d"
#   X                       = "D" 

file_name = "cta_ridership.csv"
time_dependent_variable = "total_rides"
time_column = "service_date"
frmt =  '%Y-%m-%d'
X = "D"

In [3]:
def data(time_column, file_name, frmt='%Y-%m-%d %H:%M:%S', X= "D"):
    df = pd.read_csv(file_name, parse_dates= True)
    df = df[[time_column,time_dependent_variable]]
    df[time_column] = pd.to_datetime(df[time_column],format=frmt) 
    df.index = df[time_column]
    df = df.resample(X).mean()
    df.reset_index(inplace= True)
    return df
df = data(time_column, file_name, frmt, X)

In [4]:
df

,service_date,total_rides
0,2001-01-01,423647.0
1,2001-01-02,1282779.0
2,2001-01-03,1361355.0
3,2001-01-04,1420032.0
4,2001-01-05,1448343.0
...,...,...
6934,2019-12-27,998033.0
6935,2019-12-28,693515.0
6936,2019-12-29,566533.0
6937,2019-12-30,1025434.0


In [5]:
df["time_idx"] = df.index
df["group"] = 1
df

,service_date,total_rides,time_idx,group
0,2001-01-01,423647.0,0,1
1,2001-01-02,1282779.0,1,1
2,2001-01-03,1361355.0,2,1
3,2001-01-04,1420032.0,3,1
4,2001-01-05,1448343.0,4,1
...,...,...,...,...
6934,2019-12-27,998033.0,6934,1
6935,2019-12-28,693515.0,6935,1
6936,2019-12-29,566533.0,6936,1
6937,2019-12-30,1025434.0,6937,1


In [6]:
# create the dataset from the pandas dataframe
max_prediction_length = 1000
max_encoder_length = 24
training_cutoff = df["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    group_ids=["group"],
    target=time_dependent_variable,
    time_idx="time_idx",
    min_encoder_length=2,
    max_encoder_length=6,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=[time_dependent_variable],
)

In [8]:
training.get_parameters()

{'time_idx': 'time_idx',
 'target': 'total_rides',
 'group_ids': ['group'],
 'weight': None,
 'max_encoder_length': 6,
 'min_encoder_length': 2,
 'min_prediction_idx': 0,
 'min_prediction_length': 1000,
 'max_prediction_length': 1000,
 'static_categoricals': [],
 'static_reals': ['encoder_length'],
 'time_varying_known_categoricals': [],
 'time_varying_known_reals': [],
 'time_varying_unknown_categoricals': [],
 'time_varying_unknown_reals': ['total_rides'],
 'variable_groups': {},
 'constant_fill_strategy': {},
 'allow_missing_timesteps': False,
 'lags': {},
 'add_relative_time_idx': False,
 'add_target_scales': False,
 'add_encoder_length': True,
 'target_normalizer': GroupNormalizer(transformation='relu'),
 'categorical_encoders': {'__group_id__group': NaNLabelEncoder(),
  'group': NaNLabelEncoder()},
 'scalers': {'encoder_length': StandardScaler()},
 'randomize_length': None,
 'predict_mode': False}

In [9]:
# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

In [10]:
# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=8)

In [11]:
trainer = pl.Trainer(
    gpus=0,
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Number of parameters in network: 15.3k


In [12]:
# find optimal learning rate
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

C:\Users\Agrim.rustagi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\data_loading.py:428: UserWarning: The number of training samples (38) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'item'